Bert QA

In [ ]:
!pip install -q kaggle
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"huangspray","key":"6fa5bed5ff41875d1a56a2ea015c644b"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                          title                                           size  lastUpdated          downloadCount  
-----------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  
szymonjanowski/internet-articles-data-with-users-engagement  Internet news data with readers engagement       3MB  2020-11-21 17:09:57           2415  
sakshigoyal7/credit-card-customers                           Credit Card customers                          379KB  2020-11-19 07:38:44           9135  
imoore/2020-us-general-election-turnout-rates                2020 US General Election Turnout rates           4KB  2020-11-26 00:21:15           2922  
afrniomelo/3w-dataset                                        3W Dataset - Undesirable events in oil wells   658MB  2020-11-21 21:22:49            452  
mrmorj/us-politicians-twitter-dataset                        US Politicians Twitter Data

In [ ]:
!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge

100% 6.26G/6.26G [02:18<00:00, 28.6MB/s]
100% 6.26G/6.26G [02:18<00:00, 48.4MB/s]


In [ ]:
!unzip CORD-19-research-challenge.zip -d CORD-19-research-challenge

Streaming output truncated to the last 5000 lines.
  inflating: CORD-19-research-challenge/document_parses/pmc_json/PMC7675279.xml.json  
  inflating: CORD-19-research-challenge/document_parses/pmc_json/PMC7675280.xml.json  
  inflating: CORD-19-research-challenge/document_parses/pmc_json/PMC7675281.xml.json  
  inflating: CORD-19-research-challenge/document_parses/pmc_json/PMC7675282.xml.json  
  inflating: CORD-19-research-challenge/document_parses/pmc_json/PMC7675286.xml.json  
  inflating: CORD-19-research-challenge/document_parses/pmc_json/PMC7675288.xml.json  
  inflating: CORD-19-research-challenge/document_parses/pmc_json/PMC7675290.xml.json  
  inflating: CORD-19-research-challenge/document_parses/pmc_json/PMC7675291.xml.json  
  inflating: CORD-19-research-challenge/document_parses/pmc_json/PMC7675292.xml.json  
  inflating: CORD-19-research-challenge/document_parses/pmc_json/PMC7675293.xml.json  
  inflating: CORD-19-research-challenge/document_parses/pmc_json/PMC7675296.xml

In [ ]:
##Install the packages 
!pip install sentence-transformers

     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 1.3MB 11.1MB/s 
     |████████████████████████████████| 2.9MB 27.0MB/s 
     |████████████████████████████████| 1.1MB 59.9MB/s 
     |████████████████████████████████| 890kB 51.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101034 sha256=6f15df6c2d9e1ab4459619b1455641333d658b5dfa39c2b572643a6a5742fa7e
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f4487366ddb03b520f20dc95214683e2ccd9ec3c6ec70ca12092d9c17952467e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
import os
import pandas as pd
import numpy as np
import os
import json
import warnings

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import nltk

In [ ]:
from tqdm import tqdm
def make_dataset(json_filePaths):
  dataframe = pd.DataFrame([], columns=[
        "doc_id", 
        "title", 
        "abstract", 
        "text_body"
    ])
  
  for path in json_filePaths:
      with open(path) as json_data:
        data = json.load(json_data)
        row = {
            "doc_id": None, 
            "title": None,
            "abstract": None, 
            "text_body": None
            }
        row['doc_id'] = data['paper_id']
        row['title'] = data['metadata']

        if 'abstract' in data:
          row['abstract'] = "\n ".join([abstract['text'] for abstract in data['abstract']])
        row['text_body'] = "\n ".join([textbody['text'] for textbody in data['body_text']])
                                    
      dataframe = dataframe.append(row, ignore_index=True)

  return dataframe

In [ ]:
meta = pd.read_csv("CORD-19-research-challenge/metadata.csv")
print(meta.shape)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(381817, 19)


In [ ]:
!ls

CORD-19-research-challenge	kaggle.json
CORD-19-research-challenge.zip	sample_data


In [ ]:
### first filter by meta file. select only papers after 2020
meta["publish_time"] = pd.to_datetime(meta["publish_time"])
meta["publish_year"] = (pd.DatetimeIndex(meta['publish_time']).year)
meta["publish_month"] = (pd.DatetimeIndex(meta['publish_time']).month)
meta = meta[meta["publish_year"] == 2020]
print(meta.shape[0], " papers are available after 2020 Jan 1.")

287099  papers are available after 2020 Jan 1.


In [ ]:
paths = []
count = 0
for i in range(len(meta)):
    if type(meta.iloc[i, 15])== float:
        count += 1
    else:
      if ";" not in meta.iloc[i, 15]:
        paths.append('CORD-19-research-challenge/' + meta.iloc[i, 15])
print(paths[0:10])

['CORD-19-research-challenge/document_parses/pdf_json/44449ad1cca160ce491d7624f8ae1028f3570c45.json', 'CORD-19-research-challenge/document_parses/pdf_json/def41c08c3cb1b3752bcff34d3aed7f8486e1c86.json', 'CORD-19-research-challenge/document_parses/pdf_json/f5ae3f66face323615df39d838e056ab5fcc98df.json', 'CORD-19-research-challenge/document_parses/pdf_json/5be75ae4e7f8c892abd8dc396b9dbd035772c84a.json', 'CORD-19-research-challenge/document_parses/pdf_json/1cee4a0d0e823379ec34a462a04561bf4cd736a2.json', 'CORD-19-research-challenge/document_parses/pdf_json/dfbc39af0a5845b6f3698657c81a4527a45a4021.json', 'CORD-19-research-challenge/document_parses/pdf_json/2caf288d2b6723ea0d82657ab1ecb1199a5b3b6b.json', 'CORD-19-research-challenge/document_parses/pdf_json/81f4c4710b9844ee1b4981af6e45fe286e44caa8.json', 'CORD-19-research-challenge/document_parses/pdf_json/d6430e4ecda76d74dc295a831b9fc1914f7c04db.json', 'CORD-19-research-challenge/document_parses/pdf_json/778903e91db14b0973295cc8e58a6cd4c7a13

In [ ]:
##Create dataset
data = make_dataset(paths)
print(data.head(5))


                                     doc_id  ...                                          text_body
0  44449ad1cca160ce491d7624f8ae1028f3570c45  ...  Dexmedetomidine is a sedative drug that has a ...
1  def41c08c3cb1b3752bcff34d3aed7f8486e1c86  ...  Transpulmonary thermodilution is commonly used...
2  f5ae3f66face323615df39d838e056ab5fcc98df  ...  Human metapneumovirus (HMPV) is a single-stran...
3  5be75ae4e7f8c892abd8dc396b9dbd035772c84a  ...  Antimicrobial resistance (AMR) is a threat to ...
4  1cee4a0d0e823379ec34a462a04561bf4cd736a2  ...  Carbohydrate-based vaccines have a long histor...

[5 rows x 4 columns]


In [ ]:
warnings.filterwarnings("ignore")
##bert model 1
model = SentenceTransformer('bert-base-nli-mean-tokens')

##questions from task 2
question_embedding = model.encode(['What do we know about virus genetics, origin, and evolution?'])

queries = ['What is known about transmission, incubation, and environmental stability?', 'What do we know about COVID-19 risk factors?', 
           'What do we know about virus genetics, origin, and evolution?', 'What do we know about vaccines and therapeutics?',
           'Are there geographic variations in the rate of COVID-19 spread?', 'Are there geographic variations in the mortality rate of COVID-19?',
           'Is there any evidence to suggest geographic based virus mutations?','What do we know about diagnostics and surveillance?',
           'What do we know about non-pharmaceutical interventions?','What has been published about medical care?',
           'What has been published about ethical and social science considerations?', 'What has been published about information sharing and inter-sectoral collaboration?']

query_embeddings = model.encode(queries)

100%|██████████| 405M/405M [00:29<00:00, 13.6MB/s]


In [ ]:
data['encode'] = data['abstract'].str.replace('\d+', '')
data.reset_index(drop = True, inplace = True)

In [ ]:
embeddings = model.encode(data['encode'])

In [ ]:
import scipy as sc
closest_n = 5
prediction1 = []
for query, query_embedding in zip(queries, query_embeddings):
    distances = sc.spatial.distance.cdist([query_embedding], embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
      prediction1.append(data['encode'][idx].strip())
      print(data['encode'][idx].strip(), "\n(Score: %.4f)" % (1-distance),"\n")





Query: What is known about transmission, incubation, and environmental stability?

Top 5 most similar sentences in corpus:
The brief article deals with the following questions: Was the adaptive toolbox of heuristics ecologically rational and specifically accurate in the initial stages of COVID-, which was characterized by epistemic uncertainty? In other words, in dealing with COVID- did the environmental structural variables allow the success of a given heuristic strategy? 
(Score: 0.7528) 

The idea of decoupling "environmental bads" from "economic goods" has been proposed as a path towards sustainability by organizations such as the OECD and UN. Scientific consensus reports on environmental impacts (e.g., greenhouse gas emissions) and resource use give an indication of the kind of decoupling needed for ecological sustainability: global, absolute, fast-enough and long-enough. This goal gives grounds for a categorisation of the different kinds of decoupling, with regard to their re

In [ ]:

##bert model 2
model2 = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

##questions from task 2
question_embedding2 = model2.encode(['What do we know about virus genetics, origin, and evolution?'])

query_embeddings2 = model2.encode(queries)


100%|██████████| 245M/245M [00:25<00:00, 9.48MB/s]


In [ ]:
embeddings2 = model2.encode(data['encode'])

In [ ]:
prediction2 = []
for query, query_embedding in zip(queries, query_embeddings2):
    distances = sc.spatial.distance.cdist([query_embedding], embeddings2, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
      prediction2.append(data['encode'][idx].strip())
      print(data['encode'][idx].strip(), "\n(Score: %.4f)" % (1-distance),"\n")





Query: What is known about transmission, incubation, and environmental stability?

Top 5 most similar sentences in corpus:
One of the strategies to reduce the environmental impacts associated with electricity is to employ renewable resources such as biomass or even waste. However, the evaluation of the sustainability of a power plant depends on the development of several analyses, which should encompass thermodynamic and environmental parameters. Energy, exergy, and exergoenvironmental assessments are carried out for a sugarcane bagasse cogeneration system, along with a Life Cycle Assessment for the Brazilian sugarcane bagasse, employing the Ecoindicator  method. The specific environmental impacts of electricity and steam are . mPt/MJ and . mPt/MJ, respectively, and the boiler feed pump and radiator presented the highest average environmental impact per exergy of fuel and product, respectively. The component with the highest exergoenvironmental factor was the furnace (.%), demonstr

In [ ]:
##bert model 3
model3 = SentenceTransformer('bert-base-nli-mean-tokens')

##questions from task 2
question_embedding3 = model3.encode(['What do we know about virus genetics, origin, and evolution?'])

query_embeddings3 = model3.encode(queries)

In [ ]:
embeddings3 = model3.encode(data['encode'])

In [ ]:
prediction3 = []
for query, query_embedding in zip(queries, query_embeddings3):
    distances = sc.spatial.distance.cdist([query_embedding], embeddings3, "euclidean")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
      prediction3.append(data['encode'][idx].strip())
      print(data['encode'][idx].strip(), "\n(Score: %.4f)" % (distance),"\n")





Query: What is known about transmission, incubation, and environmental stability?

Top 5 most similar sentences in corpus:
The brief article deals with the following questions: Was the adaptive toolbox of heuristics ecologically rational and specifically accurate in the initial stages of COVID-, which was characterized by epistemic uncertainty? In other words, in dealing with COVID- did the environmental structural variables allow the success of a given heuristic strategy? 
(Score: 11.0426) 

The idea of decoupling "environmental bads" from "economic goods" has been proposed as a path towards sustainability by organizations such as the OECD and UN. Scientific consensus reports on environmental impacts (e.g., greenhouse gas emissions) and resource use give an indication of the kind of decoupling needed for ecological sustainability: global, absolute, fast-enough and long-enough. This goal gives grounds for a categorisation of the different kinds of decoupling, with regard to their r

In [ ]:
##bert model 4
model4 = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

##questions from task 2
question_embedding4 = model4.encode(['What do we know about virus genetics, origin, and evolution?'])

query_embeddings4 = model4.encode(queries)

In [ ]:
embeddings4 = model4.encode(data['encode'])

In [ ]:
prediction4 = []
for query, query_embedding in zip(queries, query_embeddings4):
    distances = sc.spatial.distance.cdist([query_embedding], embeddings4, "euclidean")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
      prediction4.append(data['encode'][idx].strip())
      print(data['encode'][idx].strip(), "\n(Score: %.4f)" % (distance),"\n")





Query: What is known about transmission, incubation, and environmental stability?

Top 5 most similar sentences in corpus:
Please cite this article as: Shahbaz, M.A., Martikainen, M.-V., Rönkkö, T.J., Komppula, M., Jalava, P.I., Roponen, M., Urban air PM modifies differently immune defense responses against bacterial and viral infections in vitro, Environmental Research, https://doi.
 Epidemiological evidence has shown the association between exposure to ambient fine particulate matter (PM) and increased susceptibility to bacterial and viral respiratory infections. However, to date, the underlying mechanisms of immunomodulatory effects of PM remain unclear. Our objective was to explore how exposure to relatively low doses of urban air PM alters innate responses to bacterial and viral stimuli in vitro. We used secondary alveolar epithelial cell line along with monocyte-derived macrophages to replicate innate lung barrier in vitro. Co-cultured cells were first exposed for  hours to P

In [ ]:
print(prediction1[0:5])
print(prediction2[0:5])
print(prediction3[0:5])
print(prediction4[0:5])

['The brief article deals with the following questions: Was the adaptive toolbox of heuristics ecologically rational and specifically accurate in the initial stages of COVID-, which was characterized by epistemic uncertainty? In other words, in dealing with COVID- did the environmental structural variables allow the success of a given heuristic strategy?', 'The idea of decoupling "environmental bads" from "economic goods" has been proposed as a path towards sustainability by organizations such as the OECD and UN. Scientific consensus reports on environmental impacts (e.g., greenhouse gas emissions) and resource use give an indication of the kind of decoupling needed for ecological sustainability: global, absolute, fast-enough and long-enough. This goal gives grounds for a categorisation of the different kinds of decoupling, with regard to their relevance. We conducted a survey of recent (-) research on decoupling on Web of Science and reviewed the results in the research according to t

In [ ]:
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import download
download('stopwords')

stopwords = stopwords.words('english')

start = 0
total_dist = 0 
training_sentences = []
for string in prediction1:
  training_sentences.append(string.split())
similar_model = Word2Vec(training_sentences)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
total_dist = 0
for query in queries:
    sentence_a = ''
    sentence_b = ''
    for i in range(closest_n):
        sentence_a += prediction1[i+start]
        sentence_b += prediction2[i+start]
    sentence_a = [w for w in sentence_a if w not in stopwords]
    sentence_b = [w for w in sentence_b if w not in stopwords]
    dist = similar_model.wmdistance(sentence_a,sentence_b)
    print(query+"\n" + str(dist))
    total_dist += dist
print(total_dist)

What is known about transmission, incubation, and environmental stability?
0.0017691736726606895
What do we know about COVID-19 risk factors?
0.0017691736726606895
What do we know about virus genetics, origin, and evolution?
0.0017691736726606895
What do we know about vaccines and therapeutics?
0.0017691736726606895
Are there geographic variations in the rate of COVID-19 spread?
0.0017691736726606895
Are there geographic variations in the mortality rate of COVID-19?
0.0017691736726606895
Is there any evidence to suggest geographic based virus mutations?
0.0017691736726606895
What do we know about diagnostics and surveillance?
0.0017691736726606895
What do we know about non-pharmaceutical interventions?
0.0017691736726606895
What has been published about medical care?
0.0017691736726606895
What has been published about ethical and social science considerations?
0.0017691736726606895
What has been published about information sharing and inter-sectoral collaboration?
0.0017691736726606895

In [ ]:
for query in queries:
    sentence_a = ''
    sentence_b = ''
    for i in range(closest_n):
        sentence_a += prediction2[i+start]
        sentence_b += prediction3[i+start]
    sentence_a = [w for w in sentence_a if w not in stopwords]
    sentence_b = [w for w in sentence_b if w not in stopwords]
    dist = similar_model.wmdistance(sentence_a,sentence_b)
    print(query+"\n" + str(dist))
    total_dist += dist
print(total_dist)

What is known about transmission, incubation, and environmental stability?
0.0025609791975763595
What do we know about COVID-19 risk factors?
0.0025609791975763595
What do we know about virus genetics, origin, and evolution?
0.0025609791975763595
What do we know about vaccines and therapeutics?
0.0025609791975763595
Are there geographic variations in the rate of COVID-19 spread?
0.0025609791975763595
Are there geographic variations in the mortality rate of COVID-19?
0.0025609791975763595
Is there any evidence to suggest geographic based virus mutations?
0.0025609791975763595
What do we know about diagnostics and surveillance?
0.0025609791975763595
What do we know about non-pharmaceutical interventions?
0.0025609791975763595
What has been published about medical care?
0.0025609791975763595
What has been published about ethical and social science considerations?
0.0025609791975763595
What has been published about information sharing and inter-sectoral collaboration?
0.0025609791975763595

In [ ]:
total_dist = 0
for query in queries:
    sentence_a = ''
    sentence_b = ''
    for i in range(closest_n):
        sentence_a += prediction2[i+start]
        sentence_b += prediction4[i+start]
    sentence_a = [w for w in sentence_a if w not in stopwords]
    sentence_b = [w for w in sentence_b if w not in stopwords]
    dist = similar_model.wmdistance(sentence_a,sentence_b)
    print(query+"\n" + str(dist))
    total_dist += dist
print(total_dist)

What is known about transmission, incubation, and environmental stability?
0.0034909309213907573
What do we know about COVID-19 risk factors?
0.0034909309213907573
What do we know about virus genetics, origin, and evolution?
0.0034909309213907573
What do we know about vaccines and therapeutics?
0.0034909309213907573
Are there geographic variations in the rate of COVID-19 spread?
0.0034909309213907573
Are there geographic variations in the mortality rate of COVID-19?
0.0034909309213907573
Is there any evidence to suggest geographic based virus mutations?
0.0034909309213907573
What do we know about diagnostics and surveillance?
0.0034909309213907573
What do we know about non-pharmaceutical interventions?
0.0034909309213907573
What has been published about medical care?
0.0034909309213907573
What has been published about ethical and social science considerations?
0.0034909309213907573
What has been published about information sharing and inter-sectoral collaboration?
0.0034909309213907573

In [ ]:
total_dist = 0
for query in queries:
    sentence_a = ''
    sentence_b = ''
    for i in range(closest_n):
        sentence_a += prediction3[i+start]
        sentence_b += prediction4[i+start]
    sentence_a = [w for w in sentence_a if w not in stopwords]
    sentence_b = [w for w in sentence_b if w not in stopwords]
    dist = similar_model.wmdistance(sentence_a,sentence_b)
    print(query+"\n" + str(dist))
    total_dist += dist
print(total_dist)

What is known about transmission, incubation, and environmental stability?
0.0021565989378322173
What do we know about COVID-19 risk factors?
0.0021565989378322173
What do we know about virus genetics, origin, and evolution?
0.0021565989378322173
What do we know about vaccines and therapeutics?
0.0021565989378322173
Are there geographic variations in the rate of COVID-19 spread?
0.0021565989378322173
Are there geographic variations in the mortality rate of COVID-19?
0.0021565989378322173
Is there any evidence to suggest geographic based virus mutations?
0.0021565989378322173
What do we know about diagnostics and surveillance?
0.0021565989378322173
What do we know about non-pharmaceutical interventions?
0.0021565989378322173
What has been published about medical care?
0.0021565989378322173
What has been published about ethical and social science considerations?
0.0021565989378322173
What has been published about information sharing and inter-sectoral collaboration?
0.0021565989378322173

In [ ]:
training_sentences2 = []
for string in prediction2:
  training_sentences2.append(string.split())
similar_model2 = Word2Vec(training_sentences2)

total_dist = 0
for query in queries:
    sentence_a = ''
    sentence_b = ''
    for i in range(closest_n):
        sentence_a += prediction1[i+start]
        sentence_b += prediction3[i+start]
    sentence_a = [w for w in sentence_a if w not in stopwords]
    sentence_b = [w for w in sentence_b if w not in stopwords]
    dist = similar_model2.wmdistance(sentence_a,sentence_b)
    print(query+"\n" + str(dist))
    total_dist += dist
print(total_dist)

What is known about transmission, incubation, and environmental stability?
0.006897947432780222
What do we know about COVID-19 risk factors?
0.006897947432780222
What do we know about virus genetics, origin, and evolution?
0.006897947432780222
What do we know about vaccines and therapeutics?
0.006897947432780222
Are there geographic variations in the rate of COVID-19 spread?
0.006897947432780222
Are there geographic variations in the mortality rate of COVID-19?
0.006897947432780222
Is there any evidence to suggest geographic based virus mutations?
0.006897947432780222
What do we know about diagnostics and surveillance?
0.006897947432780222
What do we know about non-pharmaceutical interventions?
0.006897947432780222
What has been published about medical care?
0.006897947432780222
What has been published about ethical and social science considerations?
0.006897947432780222
What has been published about information sharing and inter-sectoral collaboration?
0.006897947432780222
0.082775369

In [ ]:
total_dist = 0
for query in queries:
    sentence_a = ''
    sentence_b = ''
    for i in range(closest_n):
        sentence_a += prediction1[i+start]
        sentence_b += prediction4[i+start]
    sentence_a = [w for w in sentence_a if w not in stopwords]
    sentence_b = [w for w in sentence_b if w not in stopwords]
    dist = similar_model2.wmdistance(sentence_a,sentence_b)
    print(query+"\n" + str(dist))
    total_dist += dist
print(total_dist)

What is known about transmission, incubation, and environmental stability?
0.007242692026664652
What do we know about COVID-19 risk factors?
0.007242692026664652
What do we know about virus genetics, origin, and evolution?
0.007242692026664652
What do we know about vaccines and therapeutics?
0.007242692026664652
Are there geographic variations in the rate of COVID-19 spread?
0.007242692026664652
Are there geographic variations in the mortality rate of COVID-19?
0.007242692026664652
Is there any evidence to suggest geographic based virus mutations?
0.007242692026664652
What do we know about diagnostics and surveillance?
0.007242692026664652
What do we know about non-pharmaceutical interventions?
0.007242692026664652
What has been published about medical care?
0.007242692026664652
What has been published about ethical and social science considerations?
0.007242692026664652
What has been published about information sharing and inter-sectoral collaboration?
0.007242692026664652
0.086912304

In [ ]:
total_dist = 0
for query in queries:
    sentence_a = ''
    sentence_b = ''
    for i in range(closest_n):
        sentence_a += prediction3[i+start]
        sentence_b += prediction4[i+start]
    sentence_a = [w for w in sentence_a if w not in stopwords]
    sentence_b = [w for w in sentence_b if w not in stopwords]
    dist = similar_model2.wmdistance(sentence_a,sentence_b)
    print(query+"\n" + str(dist))
    total_dist += dist
print(total_dist)

What is known about transmission, incubation, and environmental stability?
0.007105263699322713
What do we know about COVID-19 risk factors?
0.007105263699322713
What do we know about virus genetics, origin, and evolution?
0.007105263699322713
What do we know about vaccines and therapeutics?
0.007105263699322713
Are there geographic variations in the rate of COVID-19 spread?
0.007105263699322713
Are there geographic variations in the mortality rate of COVID-19?
0.007105263699322713
Is there any evidence to suggest geographic based virus mutations?
0.007105263699322713
What do we know about diagnostics and surveillance?
0.007105263699322713
What do we know about non-pharmaceutical interventions?
0.007105263699322713
What has been published about medical care?
0.007105263699322713
What has been published about ethical and social science considerations?
0.007105263699322713
What has been published about information sharing and inter-sectoral collaboration?
0.007105263699322713
0.085263164

In [ ]:
total_dist = 0
for query in queries:
    sentence_a = ''
    sentence_b = ''
    for i in range(closest_n):
        sentence_a += prediction1[i+start]
        sentence_b += prediction2[i+start]
    sentence_a = [w for w in sentence_a if w not in stopwords]
    sentence_b = [w for w in sentence_b if w not in stopwords]
    dist = similar_model2.wmdistance(sentence_a,sentence_b)
    print(query+"\n" + str(dist))
    total_dist += dist
print(total_dist)

What is known about transmission, incubation, and environmental stability?
0.006520457973208825
What do we know about COVID-19 risk factors?
0.006520457973208825
What do we know about virus genetics, origin, and evolution?
0.006520457973208825
What do we know about vaccines and therapeutics?
0.006520457973208825
Are there geographic variations in the rate of COVID-19 spread?
0.006520457973208825
Are there geographic variations in the mortality rate of COVID-19?
0.006520457973208825
Is there any evidence to suggest geographic based virus mutations?
0.006520457973208825
What do we know about diagnostics and surveillance?
0.006520457973208825
What do we know about non-pharmaceutical interventions?
0.006520457973208825
What has been published about medical care?
0.006520457973208825
What has been published about ethical and social science considerations?
0.006520457973208825
What has been published about information sharing and inter-sectoral collaboration?
0.006520457973208825
0.078245495